In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Print the path to confirm
print("Google Drive Mounted Path:", os.listdir('/content/drive'))


Google Drive Mounted Path: ['MyDrive', '.shortcut-targets-by-id', '.file-revisions-by-id', '.Trash-0', '.Encrypted']


In [4]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import os


drive_path = "/content/drive/MyDrive/NN_Hyperparameter_Results/"

if not os.path.exists(drive_path):
    os.makedirs(drive_path)


output_file = os.path.join(drive_path, "nn_hyperparameter_results.xlsx")


if not os.path.exists(output_file):
    results = pd.DataFrame(columns=[
        "Configuration No.", "Output", "Loss Curve", "Accuracy Curve"
    ])
    results.to_excel(output_file, index=False)
    print(f"Excel file initialized: {output_file}")

# Disable eager execution for TensorFlow v1 compatibility
tf.compat.v1.disable_eager_execution()

# Loading MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0  # Normalize

#  Reshape into vectors
X_train = X_train.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

# One-hot encode the labels
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

#Hyperparameters
learning_rate = 0.1
epochs = 50
batch_size = 10

#  Hyperparameter combinations
activations = ['relu', 'sigmoid', 'tanh']
hidden_layers = [256, 128, 64]

# Function to create and train the model
def train_and_evaluate(X_train, y_train, X_test, y_test, activation, hidden_layer_size, lr, bs, ep):
    tf.compat.v1.reset_default_graph()

    # Placeholders
    X = tf.compat.v1.placeholder(tf.float32, [None, 28 * 28])
    Y = tf.compat.v1.placeholder(tf.float32, [None, 10])

    # Hidden layer
    W1 = tf.Variable(tf.random.normal([28 * 28, hidden_layer_size]))
    b1 = tf.Variable(tf.random.normal([hidden_layer_size]))

    if activation == "relu":
        hidden_output = tf.nn.relu(tf.matmul(X, W1) + b1)
    elif activation == "sigmoid":
        hidden_output = tf.nn.sigmoid(tf.matmul(X, W1) + b1)
    elif activation == "tanh":
        hidden_output = tf.nn.tanh(tf.matmul(X, W1) + b1)

    # Output layer
    W2 = tf.Variable(tf.random.normal([hidden_layer_size, 10]))
    b2 = tf.Variable(tf.random.normal([10]))

    logits = tf.matmul(hidden_output, W2) + b2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

    optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)

    # Accuracy calculation
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Training
    init = tf.compat.v1.global_variables_initializer()

    loss_curve = []
    acc_curve = []

    start_time = time.time()

    with tf.compat.v1.Session() as sess:
        sess.run(init)

        num_batches = len(X_train) // bs
        for epoch in range(ep):
            for i in range(num_batches):
                batch_x = X_train[i * bs:(i + 1) * bs]
                batch_y = y_train[i * bs:(i + 1) * bs]
                sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y})

            # Loss and accuracy for each epoch
            epoch_loss, epoch_acc = sess.run([loss, accuracy], feed_dict={X: X_train, Y: y_train})

            loss_curve.append(epoch_loss)
            acc_curve.append(epoch_acc)

            print(f"Epoch {epoch + 1}/{ep}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

        # Test accuracy and confusion matrix
        test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
        predictions = sess.run(tf.argmax(logits, 1), feed_dict={X: X_test})
        true_labels = np.argmax(y_test, axis=1)

        conf_matrix = confusion_matrix(true_labels, predictions)
        execution_time = time.time() - start_time


        plt.figure(figsize=(12, 5))

        # Loss curve
        plt.subplot(1, 2, 1)
        plt.plot(loss_curve, label='Loss', color='blue')
        plt.title(f"Loss Curve - {activation} - {hidden_layer_size}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        loss_curve_img = f"{drive_path}loss_{activation}_{hidden_layer_size}.png"
        plt.savefig(loss_curve_img)

        # Accuracy curve
        plt.subplot(1, 2, 2)
        plt.plot(acc_curve, label='Accuracy', color='green')
        plt.title(f"Accuracy Curve - {activation} - {hidden_layer_size}")
        plt.xlabel("Epochs")
        plt.ylabel("Accuracy")
        plt.legend()
        acc_curve_img = f"{drive_path}accuracy_{activation}_{hidden_layer_size}.png"
        plt.savefig(acc_curve_img)

        plt.close()


        output_text = (
            f"Results for Combination:\n"
            f"Activation: {activation}\n"
            f"Hidden Layer: {hidden_layer_size}\n"
            f"Test Accuracy: {test_acc:.4f}\n"
            f"Execution Time: {execution_time:.2f} sec\n"
            f"Confusion Matrix:\n{conf_matrix}"
        )


        print("\n" + output_text + "\n")

        return {
            "output": output_text,
            "loss_curve": loss_curve_img,
            "accuracy_curve": acc_curve_img
        }

# Iterating through all combinations
config_no = 1
for activation in activations:
    for hidden_layer in hidden_layers:
        print("\n--------------------------------------------------")
        print(f"Training with Activation: {activation}, Hidden Layer: {hidden_layer}")
        print(f"Learning Rate: {learning_rate}, Batch Size: {batch_size}, Epochs: {epochs}")
        print("--------------------------------------------------")

        results_dict = train_and_evaluate(
            X_train, y_train, X_test, y_test,
            activation, hidden_layer, learning_rate, batch_size, epochs
        )


        if os.path.exists(output_file):
            results = pd.read_excel(output_file)
        else:
            results = pd.DataFrame(columns=[
                "Configuration No.", "Output", "Loss Curve", "Accuracy Curve"
            ])

        new_row = pd.DataFrame([{
            "Configuration No.": config_no,
            "Output": results_dict["output"],
            "Loss Curve": results_dict["loss_curve"],
            "Accuracy Curve": results_dict["accuracy_curve"]
        }])

        results = pd.concat([results, new_row], ignore_index=True)


        results.to_excel(output_file, index=False)

        config_no += 1

print("\n All combinations tested. Results are saved:", output_file)



--------------------------------------------------
Training with Activation: relu, Hidden Layer: 256
Learning Rate: 0.1, Batch Size: 10, Epochs: 50
--------------------------------------------------
Epoch 1/50, Loss: 0.4243, Accuracy: 0.9175
Epoch 2/50, Loss: 0.2404, Accuracy: 0.9435
Epoch 3/50, Loss: 0.1899, Accuracy: 0.9520
Epoch 4/50, Loss: 0.1487, Accuracy: 0.9595
Epoch 5/50, Loss: 0.1195, Accuracy: 0.9650
Epoch 6/50, Loss: 0.1042, Accuracy: 0.9686
Epoch 7/50, Loss: 0.0857, Accuracy: 0.9734
Epoch 8/50, Loss: 0.0745, Accuracy: 0.9768
Epoch 9/50, Loss: 0.0699, Accuracy: 0.9774
Epoch 10/50, Loss: 0.0585, Accuracy: 0.9804
Epoch 11/50, Loss: 0.0485, Accuracy: 0.9837
Epoch 12/50, Loss: 0.0465, Accuracy: 0.9847
Epoch 13/50, Loss: 0.0398, Accuracy: 0.9867
Epoch 14/50, Loss: 0.0366, Accuracy: 0.9881
Epoch 15/50, Loss: 0.0306, Accuracy: 0.9894
Epoch 16/50, Loss: 0.0290, Accuracy: 0.9906
Epoch 17/50, Loss: 0.0232, Accuracy: 0.9923
Epoch 18/50, Loss: 0.0193, Accuracy: 0.9936
Epoch 19/50, Loss